# Custom Training StyleGan2-ADA

I chose [StyleGAN2-ADA by NVIDA](https://github.com/NVlabs/stylegan2) to generate the images for this tarot deck based on examples I found online and the availability of tutorials and reference materials. 

This tutorial by [Derrick Schultz](https://artificial-images.com/) on YouTube was especially helpful and I followed it very closely. I did ended up tweaking some of the training parameters in the `train.py` file, and those notes can be found below.

I'll only be sharing this main notebook used to train and generate images - for the full repo required to run this, please check out [Derrick's forked version of NVIDIA's StyleGAN2-ADA on GitHub](https://github.com/dvschultz/stylegan2-ada).

A word to the wise, training GANs can be an extremely fickle process. I spent the better part of two weeks running this constantly on Google Colab -- running into dead ends, tweaking, re-starting, resuming... For anyone interested in running StyleGAN2, I'd highly recommend paying the $10/month for Colab Pro: you'll get access to faster GPU's and longer run times (up to 20 hours in a session). 

StyleGAN2-ADA only work with Tensorflow 1.

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!nvidia-smi

Fri Dec  4 16:50:10 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install Repo to Google Drive

Installing [Derrick Schultz's version of the StyleGAN2-ADA repo](https://github.com/dvschultz/stylegan2-ada)


First, mounting my Google Drive to the Colab notebook: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Installing the repo during the first run of this notebook. Then simply moving into the directory for subsequent trainings. 

In [ ]:
import os
if os.path.isdir("/content/drive/My Drive/colab-sg2-ada"):
    %cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada"
else:
    #install script
    %cd "/content/drive/My Drive/"
    !mkdir colab-sg2-ada
    %cd colab-sg2-ada
    !git clone https://github.com/dvschultz/stylegan2-ada
    %cd stylegan2-ada
    !mkdir downloads
    !mkdir datasets

/content/drive/My Drive/colab-sg2-ada/stylegan2-ada


## Convert dataset to .tfrecords

**NOTE: Only done when first creating the dataset -- skip when resuming training**

First, unzipping uploaded photos to content folder in Colab

In [ ]:
# unzippng photos loaded into content folder
zip_path = "/content/tarot_512_os_2.zip"

!unzip {zip_path} -d /content/

Converting images to the `.tfrecords` format.

In [ ]:
#update this to the path to your image folder
dataset_path = "/content/tarot_512_os_2"
#give your dataset a name
dataset_name = "tarot_os2"

#you don't need to edit anything here
!python dataset_tool.py create_from_images ./datasets/{dataset_name} {dataset_path}

Loading images from "/content/tarot_512_os_2"
Creating dataset "./datasets/tarot_os2"
Added 2978 images.


## Train a custom model

We’re ready to start training! There are numerous arguments to training, what’s listed below are the most popular options. To see all the options, run the following cell.

In [ ]:
!python train.py --help

usage: train.py [-h] --outdir DIR [--gpus INT] [--snap INT] [--seed INT] [-n]
                --data PATH [--res INT] [--mirror BOOL] [--mirrory BOOL]
                [--use-raw BOOL] [--metrics LIST] [--metricdata PATH]
                [--cfg {auto,11gb-gpu,11gb-gpu-complex,24gb-gpu,24gb-gpu-complex,48gb-gpu,stylegan2,paper256,paper512,paper1024,cifar,cifarbaseline}]
                [--gamma FLOAT] [--kimg INT] [--aug {noaug,ada,fixed,adarv}]
                [--p FLOAT] [--target TARGET]
                [--augpipe {blit,geom,color,filter,noise,cutout,bg,bgc,bgcf,bgcfn,bgcfnc}]
                [--cmethod {nocmethod,bcr,zcr,pagan,wgangp,auxrot,spectralnorm,shallowmap,adropout}]
                [--dcap FLOAT] [--resume RESUME] [--freezed INT]

Train a GAN using the techniques described in the paper
"Training Generative Adversarial Networks with Limited Data".

optional arguments:
  -h, --help            show this help message and exit

general options:
  --outdir DIR          Where to sa

In [ ]:
#this name must EXACTLY match the dataset name you used when creating the .tfrecords file
dataset_name = "tarot_os2"
#how often should the model generate samples and a .pkl file
snapshot_count = 4
#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = False
#metrics? 
metric_list = None
#augments
augs = "bg"
# Gamma
set_gamma = 35.0

#------- Resume From -------#
#
# running it for the first time? set it to ffhq(+resolution)
# resume_from = 'ffhq512'

# resuming? get the path to your latest .pkl file and use that
resume_from = "/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00020-tarot_os2-mirror-11gb-gpu-gamma35-bg-resumecustom/network-snapshot-000832.pkl"

#don't edit this unless you know what you're doing :)
!python train.py --outdir ./results --snap={snapshot_count} --cfg=11gb-gpu --data=./datasets/{dataset_name} --augpipe={augs} --mirror={mirrored} --mirrory={mirroredY} --metrics={metric_list} --resume={resume_from} --gamma={set_gamma}

tcmalloc: large alloc 4294967296 bytes == 0x7ef8000 @  0x7f6e1d604001 0x7f6e1a82d765 0x7f6e1a891bb0 0x7f6e1a893a4f 0x7f6e1a92a048 0x50a4a5 0x50cc96 0x507be4 0x508ec2 0x594a01 0x549e8f 0x5515c1 0x5a9dac 0x50a433 0x50cc96 0x507be4 0x588e5c 0x59fd0e 0x50d256 0x507be4 0x588e5c 0x59fd0e 0x50d256 0x507be4 0x588e5c 0x59fd0e 0x50d256 0x5095c8 0x50a2fd 0x50beb4 0x507be4
tcmalloc: large alloc 4294967296 bytes == 0x7f6c4b420000 @  0x7f6e1d6021e7 0x7f6e1a82d5e1 0x7f6e1a891c78 0x7f6e1a891f37 0x7f6e1a929f28 0x50a4a5 0x50cc96 0x507be4 0x509900 0x50a2fd 0x50cc96 0x507be4 0x509900 0x50a2fd 0x50cc96 0x507be4 0x509900 0x50a2fd 0x50cc96 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96 0x507be4
tcmalloc: large alloc 4294967296 bytes == 0x7f6b4a41e000 @  0x7f6e1d6021e7 0x7f6e1a82d5e1 0x7f6e1a891c78 0x7f6e1a891f37 0x7f6dd88970c5 0x7f6dd821a902 0x7f6dd821aeb2 0x7f6dd81d3c3e 0x50a12f 0x50beb4 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x588c8b 0x59fd0e 0x50d256 0x507b

I frequently ran into mode collapse issues while training with some of the standard hyperparameters. I found the best training arguments for this project were: 
* Learning Rate: 0.001 - 0.0016
* Increase batch size from 4 to 8
* Increase gamma from 10 to 30-35

### While it’s training...
**Once the above cell is running you should be training!**

Don’t close this tab! Colab needs to be open and running in order to continue training. Every ~15min or so a new line should get added to your output, indicated its still training. Depending on you `snapshot_count` setting you should see the results folder in your Google drive folder fill with both samples (`fakesXXXXXx.jpg`) and model weights (`network-snapshot-XXXXXX.pkl`). The samples are worth looking at while it trains but don’t get too worried about each individual sample.

If you chose a metric, you will also see scores for each snapshot. Don’t obsess over these! they are a guide, it can go up or down slightly for each snapshot. What you want to see is a gradual lowering of the score over time.

Once Colab shuts off, you can Reconnect the notebook and re-run every cell from top to bottom. Make sure you update the `resume_from` path to continue training from the latest model.

## Generating Images

As I mentioned, Derrick Schultz has put out a *ton* of great content on working with GANs on his [YouTube channel](https://www.youtube.com/channel/UCaZuPdmZ380SFUMKHVsv_AA). I studied [this video](https://www.youtube.com/watch?v=h0zwLMIt-hI&amp;t=1s) to play around with the following image generation options. 

In [ ]:
!pip install opensimplex

In [ ]:
pkl_path = "/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00021-tarot_os2-mirror-11gb-gpu-gamma35-bg-resumecustom/network-snapshot-000832.pkl"

out_path = "/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/images-final"


!python generate.py generate-images --network={pkl_path} --seeds=161 --outdir={out_path}

Loading networks from "/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00021-tarot_os2-mirror-11gb-gpu-gamma35-bg-resumecustom/network-snapshot-000832.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating image for seed 161 (0/1) ...


Note: I probably reviewed a thousand images to pick my final 22 for the tarot deck.

### Truncation

* Morph images from less realistic to more realistic
* less truncated = less realisitc
* can be a float
* reduces latent space

In [ ]:
pkl_path = "/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00021-tarot_os2-mirror-11gb-gpu-gamma35-bg-resumecustom/network-snapshot-000832.pkl"


!python generate.py truncation-traversal --network={pkl_path} --seed=161 --start=-2.0 --stop=2.0 --increment=0.05 --outdir='./tt' --fps=30

Loading networks from "/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00021-tarot_os2-mirror-11gb-gpu-gamma35-bg-resumecustom/network-snapshot-000832.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating truncation -2.00
Generating truncation -1.95
Generating truncation -1.90
Generating truncation -1.85
Generating truncation -1.80
Generating truncation -1.75
Generating truncation -1.70
Generating truncation -1.65
Generating truncation -1.60
Generating truncation -1.55
Generating truncation -1.50
Generating truncation -1.45
Generating truncation -1.40
Generating truncation -1.35
Generating truncation -1.30
Generating truncation -1.25
Generating truncation -1.20
Generating truncation -1.15
Generating truncation -1.10
Generating truncation -1.05
Generating truncation -1.00
Generating truncation -0.95
Generating truncation -0.90
Generating truncation -0.85
Generating truncation -0.80


### Interpolation

* Move between different points in the latent space
* make first seed = last seed to make a LOOP

In [ ]:
pkl_path = "/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00021-tarot_os2-mirror-11gb-gpu-gamma35-bg-resumecustom/network-snapshot-000832.pkl"

!python generate.py generate-latent-walk --network={pkl_path} --walk-type="line-w" --seeds=161,1605,70103,80128,161 --outdir="./w-walk2"

Loading networks from "/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00021-tarot_os2-mirror-11gb-gpu-gamma35-bg-resumecustom/network-snapshot-000832.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating image for step 0/244 ...
Generating image for step 1/244 ...
Generating image for step 2/244 ...
Generating image for step 3/244 ...
Generating image for step 4/244 ...
Generating image for step 5/244 ...
Generating image for step 6/244 ...
Generating image for step 7/244 ...
Generating image for step 8/244 ...
Generating image for step 9/244 ...
Generating image for step 10/244 ...
Generating image for step 11/244 ...
Generating image for step 12/244 ...
Generating image for step 13/244 ...
Generating image for step 14/244 ...
Generating image for step 15/244 ...
Generating image for step 16/244 ...
Generating image for step 17/244 ...
Generating image for step 18/244 ...
Generat

### Noise loop interpolation

Similar to latent walk above, but only set one starting seed and the path will be chosen for you.

In [ ]:
pkl_path = "/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00021-tarot_os2-mirror-11gb-gpu-gamma35-bg-resumecustom/network-snapshot-000832.pkl"

!python generate.py generate-latent-walk --network={pkl_path} --walk-type="noiseloop" --start_seed=161 --outdir="./noisel"

Loading networks from "/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00021-tarot_os2-mirror-11gb-gpu-gamma35-bg-resumecustom/network-snapshot-000832.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating image for step 0/240 ...
Generating image for step 1/240 ...
Generating image for step 2/240 ...
Generating image for step 3/240 ...
Generating image for step 4/240 ...
Generating image for step 5/240 ...
Generating image for step 6/240 ...
Generating image for step 7/240 ...
Generating image for step 8/240 ...
Generating image for step 9/240 ...
Generating image for step 10/240 ...
Generating image for step 11/240 ...
Generating image for step 12/240 ...
Generating image for step 13/240 ...
Generating image for step 14/240 ...
Generating image for step 15/240 ...
Generating image for step 16/240 ...
Generating image for step 17/240 ...
Generating image for step 18/240 ...
Generat

## Projection
- take an image from **outside** your dataset and find the closest approximation **in** your dataset
- if you find an image that is an exact match inside the data it probably means it was generated by the dataset
- lets you see the interaction of your dataset with data from the wild

In [ ]:
pkl_path = "/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00021-tarot_os2-mirror-11gb-gpu-gamma35-bg-resumecustom/network-snapshot-000832.pkl"

target = "/content/cbd_05.jpg"

!python projector.py --outdir="./projection" --target={target} --network={pkl_path}

Loading networks from "/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00021-tarot_os2-mirror-11gb-gpu-gamma35-bg-resumecustom/network-snapshot-000832.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Projector: Computing W midpoint and stddev using 10000 samples...
Projector: std = 16.8972
Projector: Setting up noise inputs...
Projector: Building image output graph...
Projector: Building loss graph...
Projector: Building noise regularization graph...
Projector: Setting up optimizer...
Projector: Preparing target images...
Projector: Initializing optimization state...
100% 1000/1000 [01:45<00:00,  9.47it/s, dist=0.2429, loss=0.24]


## Nearest Neighbors

- lets say you have a seed you like, but want to see other images like it to see if there's something better. Now you can with the `near-neighbor` argument
- `--network`, `--seeds`, and `truncation_psi` work the same as above.
- `--diameter`: this sets how far away from the seedyou want to generate the images. `.0000001` is really close, `0.5` is really far
- `--num_samples`: how many samples you want to produce
- `--save_vector`: this will save the vector as a file in the `.npy` format. You can then use this for interpolation (not super well supported right now, but can be used manually -- see the Projection code for an example of reading an .npy file and interpolating it)

In [ ]:
pkl_path = "/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00021-tarot_os2-mirror-11gb-gpu-gamma35-bg-resumecustom/network-snapshot-000832.pkl"


!python generate.py generate-neighbors --network={pkl_path} --seeds=161 --num_samples=5 --outdir="./neighbors" --diameter=0.0005

Loading networks from "/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00021-tarot_os2-mirror-11gb-gpu-gamma35-bg-resumecustom/network-snapshot-000832.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating image for seed 161 (1/1) ...
